## Preprocessing

**Interesting links**

https://medium.com/@cd_24/bertopic-fine-tune-parameters-76c3377016fb

**Import Libraries**

In [1]:
from bertopic import BERTopic
from hdbscan import HDBSCAN
from sklearn.cluster import KMeans
from umap import UMAP

from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
from transformers import AutoModel, AutoTokenizer


import pandas as pd
import openpyxl

**Inputs**

In [2]:
# choose from 'multi or 'seznam'
model_type = 'multi'

# choose clustering method: 'k-meand' 'db-scan'
cluster_type = 'k-means'

file_name = 'data_sport_history_chemistry_film_biology.csv'
stop_words = 'input/stop_words_czech.txt'


output_vect = f'output/results-vectorized-{model_type}-{cluster_type}.xlsx'
output_reduced = f'output/results-reduced-{model_type}-{cluster_type}.xlsx'

**Load data**

In [3]:
data = pd.read_csv('input/' + file_name, encoding='utf-8', delimiter=';')
data.head(5)

,sentences
0,Fotbal je populárním sportem ve většině zemí.
1,Tenisový turnaj Wimbledon je jedním z nejstarš...
2,"Běhání je skválý způsob, jak si udržet kondici."
3,"Hokejový zápas může být velmi napínavý, zejmén..."
4,Plavání je vynikající cvičení pro posílení sva...


**Put data in a list**

In [4]:
docs = [item for sublist in data.values.tolist() for item in sublist]
docs[:5]

['Fotbal je populárním sportem ve většině zemí.',
 'Tenisový turnaj Wimbledon je jedním z nejstarších a nejprestižnějších na světě.',
 'Běhání je skválý způsob, jak si udržet kondici.',
 'Hokejový zápas může být velmi napínavý, zejména v prodloužení.',
 'Plavání je vynikající cvičení pro posílení svalů celého těla.']

In [5]:
len(docs)

499

## Set up model

**Define the clustering model**

In [6]:
if cluster_type == 'db-scan':
    cluster_model = HDBSCAN(min_cluster_size=30, 
                            min_samples=10,
                            #gen_min_span_tree=True,
                            #prediction_data=True
                            )


elif cluster_type == 'k-means':
    #does not produce any outliers!!!
    cluster_model = KMeans(n_clusters=8)

**Define Count Vectorizer**

In [7]:
# Read your custom Czech stop words from the CSV file
stopwords_df = pd.read_csv(stop_words, header=None)
czech_stop_words = stopwords_df[0].tolist()


# Convert the set of stop words into a list
czech_stop_words_list = list(czech_stop_words)
vectorizer_model = CountVectorizer(ngram_range=(1, 3), stop_words=czech_stop_words_list)

**Define the UMAP**


In [8]:
umap_model = UMAP(n_neighbors=3, n_components=3, min_dist=0.05)

**Define Sentence Transformer**

In [9]:
 # model-multi
if model_type == 'multi':
    model_name = 'all-MiniLM-L6-v2'
    model = SentenceTransformer(model_name)

# model-Seznam
elif model_type == 'seznam':
    model_name = 'Seznam/retromae-small-cs'
    model = AutoModel.from_pretrained(model_name)

**Define model**

In [10]:
model = BERTopic(#language="czech", # not necessary if sentence transformer is chosen
                 embedding_model = model,
                 min_topic_size = 10,
                 top_n_words = 15,
                 #vectorizer_model=vectorizer_model, 
                 hdbscan_model = cluster_model, 
                 umap_model = umap_model
                 )

## Train model

In [11]:
""" It is not going to work with a few hundereds documents, better to have thousands"""

topics, probs = model.fit_transform(docs)

## Coherence score

In [12]:
#https://github.com/MaartenGr/BERTopic/issues/90

import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel
import pandas as pd


'''
if error boolean - run the whole notebook again!!!
'''

def coherence_score(model, docs: pd.DataFrame, topics):
    # Merge all sentences for certain topic
    documents = pd.DataFrame({"Document": docs,
                            #"ID": range(len(docs)),
                            "Topic": topics})

    documents_per_topic = documents.groupby(['Topic'], as_index=False).agg({'Document': ' '.join})
    cleaned_docs = model._preprocess_text(documents_per_topic.Document.values)

    # Extract vectorizer and analyzer from BERTopic
    vectorizer = model.vectorizer_model
    analyzer = vectorizer.build_analyzer()

    # Use '.get_feature_names_out()' if you get an error with '.get_feature_names()'
    words = vectorizer.get_feature_names_out()

    # Extract features for Topic Coherence evaluation
    tokens = [analyzer(doc) for doc in cleaned_docs]
    dictionary = corpora.Dictionary(tokens)
    corpus = [dictionary.doc2bow(token) for token in tokens]


    # Extract words in each topic if they are non-empty and exist in the dictionary - SIMPLE 
    topic_words = [[words for words, _ in model.get_topic(topic)] 
               for topic in range(len(set(topics))-1)]

    # Evaluate Coherence
    coherence_model = CoherenceModel(topics=topic_words, 
                                    texts=tokens, 
                                    corpus=corpus,
                                    dictionary=dictionary, 
                                    coherence='c_v')
    coherence = coherence_model.get_coherence()
    return round(coherence,3)

In [13]:
coherence_score(model=model, docs=docs, topics=topics)

0.774

## Visualize results

In [14]:
model.get_topic_freq()

,Topic,Count
3,0,131
6,1,95
4,2,82
0,3,75
2,4,72
5,5,22
1,6,11
7,7,11


In [15]:
model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,131,0_roce_byl_po_na,"[roce, byl, po, na, století, první, ve, české,...",[Třicetiletá válka v 17. století měla tragický...
1,1,95,1_jsou_organismů_organismy_zkoumá,"[jsou, organismů, organismy, zkoumá, které, dn...",[Ekologie zkoumá vztahy mezi organismy a jejic...
2,2,82,2_film_český_získal_za,"[film, český, získal, za, je, filmová, filmy, ...",[Kolya získal Oscara za nejlepší cizojazyčný f...
3,3,75,3_je_sport_sportem_který,"[je, sport, sportem, který, oblíbeným, na, pro...",[Cyklistika je oblíbeným sportem pro rekreační...
4,4,72,4_chemická_jsou_reakce_mají,"[chemická, jsou, reakce, mají, rychlosti, atom...","[Chemická rovnováha nastává, když rychlosti př..."
5,5,22,5_buněk_je_proces_prostředí,"[buněk, je, proces, prostředí, univerzit, stře...",[Karlova univerzita v Praze byla založena v ro...
6,6,11,6_atletika_turnaj_wimbledon_dálky,"[atletika, turnaj, wimbledon, dálky, skok, ten...",[Tenisový turnaj Wimbledon je považován za nej...
7,7,11,7_reakce_exotermické_hoře_bílé,"[reakce, exotermické, hoře, bílé, endotermické...",[Chemické reakce mohou být exotermické nebo en...


In [16]:
model.visualize_barchart(top_n_topics=10)

In [17]:
model.visualize_topics()

In [18]:
model.visualize_heatmap(top_n_topics=20, width=800, height=800)

## Representation - Count Vectorizer

**Update topics**

In [19]:
model.update_topics(docs, vectorizer_model=vectorizer_model)

**Get counts of the topics**

In [20]:
results_unique_topics = model.get_topic_info()
results_unique_topics[:20]

,Topic,Count,Name,Representation,Representative_Docs
0,0,131,0_roce_století_české_měla,"[roce, století, české, měla, světové, revoluce...",[Třicetiletá válka v 17. století měla tragický...
1,1,95,1_organismů_organismy_zkoumá_dna,"[organismů, organismy, zkoumá, dna, živých, ži...",[Ekologie zkoumá vztahy mezi organismy a jejic...
2,2,82,2_film_český_získal_filmová,"[film, český, získal, filmová, filmy, český fi...",[Kolya získal Oscara za nejlepší cizojazyčný f...
3,3,75,3_sport_sportem_oblíbeným_cyklistika,"[sport, sportem, oblíbeným, cyklistika, sporto...",[Cyklistika je oblíbeným sportem pro rekreační...
4,4,72,4_chemická_reakce_atomy_rychlosti,"[chemická, reakce, atomy, rychlosti, prvky, mo...","[Chemická rovnováha nastává, když rychlosti př..."
5,5,22,5_buněk_proces_prostředí_nejstarších univerzit...,"[buněk, proces, prostředí, nejstarších univerz...",[Karlova univerzita v Praze byla založena v ro...
6,6,11,6_atletika_atletika zahrnuje_skok dálky_teniso...,"[atletika, atletika zahrnuje, skok dálky, teni...",[Tenisový turnaj Wimbledon je považován za nej...
7,7,11,7_reakce_exotermické_hoře_bílé hoře,"[reakce, exotermické, hoře, bílé hoře, endoter...",[Chemické reakce mohou být exotermické nebo en...


**Coherence score**

In [21]:
coherence_score(model=model, docs=docs, topics=topics)

0.725

**Export voctorized data**

In [22]:
# get results into a df
results_vect = model.get_document_info(docs)

# Export to Excel
results_vect.to_excel(output_vect, index=False, engine='openpyxl')

## Topic reduction

In [23]:
# manually merge topics
topics_to_merge = [(7, 4),
                   (2, 5, 6)
                   ]


model.merge_topics(docs, topics_to_merge)

In [24]:
""" # Further reduce topics
new_model = model.reduce_topics(docs, nr_topics=5) """

' # Further reduce topics\nnew_model = model.reduce_topics(docs, nr_topics=5) '

In [25]:
model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,131,0_roce_století_české_měla,"[roce, století, české, měla, světové, revoluce...","[Fotosyntéza je proces, při kterém rostliny vy..."
1,1,115,1_film_český_získal_filmová,"[film, český, získal, filmová, filmy, filmu, č...",[Kolya režírovaný Janem Svěrákem vyhrál Oscara...
2,2,95,2_organismů_organismy_zkoumá_dna,"[organismů, organismy, zkoumá, dna, živých, ži...",[Ekologie zkoumá vztahy mezi organismy a jejic...
3,3,83,3_reakce_chemická_chemické_rychlosti,"[reakce, chemická, chemické, rychlosti, atomy,...","[Chemická rovnováha nastává, když rychlosti př..."
4,4,75,4_sport_sportem_oblíbeným_cyklistika,"[sport, sportem, oblíbeným, cyklistika, sporto...","[Judo je bojový sport, který zdůrazňuje techni..."


In [26]:
new_results=model.get_document_info(docs)
new_results.head(100)

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Representative_document
0,Fotbal je populárním sportem ve většině zemí.,4,4_sport_sportem_oblíbeným_cyklistika,"[sport, sportem, oblíbeným, cyklistika, sporto...","[Judo je bojový sport, který zdůrazňuje techni...",sport - sportem - oblíbeným - cyklistika - spo...,False
1,Tenisový turnaj Wimbledon je jedním z nejstarš...,1,1_film_český_získal_filmová,"[film, český, získal, filmová, filmy, filmu, č...",[Kolya režírovaný Janem Svěrákem vyhrál Oscara...,film - český - získal - filmová - filmy - film...,False
2,"Běhání je skválý způsob, jak si udržet kondici.",4,4_sport_sportem_oblíbeným_cyklistika,"[sport, sportem, oblíbeným, cyklistika, sporto...","[Judo je bojový sport, který zdůrazňuje techni...",sport - sportem - oblíbeným - cyklistika - spo...,False
3,"Hokejový zápas může být velmi napínavý, zejmén...",3,3_reakce_chemická_chemické_rychlosti,"[reakce, chemická, chemické, rychlosti, atomy,...","[Chemická rovnováha nastává, když rychlosti př...",reakce - chemická - chemické - rychlosti - ato...,False
4,Plavání je vynikající cvičení pro posílení sva...,4,4_sport_sportem_oblíbeným_cyklistika,"[sport, sportem, oblíbeným, cyklistika, sporto...","[Judo je bojový sport, který zdůrazňuje techni...",sport - sportem - oblíbeným - cyklistika - spo...,False
...,...,...,...,...,...,...,...
95,Chemická analýza může odhalit složení látek a ...,2,2_organismů_organismy_zkoumá_dna,"[organismů, organismy, zkoumá, dna, živých, ži...",[Ekologie zkoumá vztahy mezi organismy a jejic...,organismů - organismy - zkoumá - dna - živých ...,False
96,"Rovnováha v chemii se vyskytuje, když rychlost...",3,3_reakce_chemická_chemické_rychlosti,"[reakce, chemická, chemické, rychlosti, atomy,...","[Chemická rovnováha nastává, když rychlosti př...",reakce - chemická - chemické - rychlosti - ato...,False
97,Exotermické reakce uvolňují energii do okolí v...,3,3_reakce_chemická_chemické_rychlosti,"[reakce, chemická, chemické, rychlosti, atomy,...","[Chemická rovnováha nastává, když rychlosti př...",reakce - chemická - chemické - rychlosti - ato...,False
98,Chirální molekuly mají zrcadlově symetrické ve...,3,3_reakce_chemická_chemické_rychlosti,"[reakce, chemická, chemické, rychlosti, atomy,...","[Chemická rovnováha nastává, když rychlosti př...",reakce - chemická - chemické - rychlosti - ato...,False


In [27]:
new_results.to_excel(output_reduced, index=False, engine='openpyxl')

## Classification

In [28]:
'''
I manually label at least 20 documents from previous topic modeling and I use these documents as input for classification.
'''

'\nI manually label at least 20 documents from previous topic modeling and I use these documents as input for classification.\n'

![Alt text](other/classification.jpg)

In [29]:
""" https://maartengr.github.io/BERTopic/getting_started/supervised/supervised.html """

' https://maartengr.github.io/BERTopic/getting_started/supervised/supervised.html '

In [30]:
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.dimensionality import BaseDimensionalityReduction
from sklearn.linear_model import LogisticRegression

In [31]:
czech_model = SentenceTransformer("ufal/robeczech-base")

No sentence-transformers model found with name ufal/robeczech-base. Creating a new one with MEAN pooling.
Some weights of RobertaModel were not initialized from the model checkpoint at ufal/robeczech-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**Read input data and prepare them for classification**

In [32]:
# read data
df_input = pd.read_excel('input/data-classification.xlsx')
df_input.head(2)

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Representative_document
0,Fotbal je populárním sportem ve většině zemí.,2,2_sport_sportem_oblíbeným_energii,"['sport', 'sportem', 'oblíbeným', 'energii', '...","['Fotosyntéza je proces, při kterém rostliny v...",sport - sportem - oblíbeným - energii - fotosy...,False
1,Tenisový turnaj Wimbledon je jedním z nejstarš...,2,2_sport_sportem_oblíbeným_energii,"['sport', 'sportem', 'oblíbeným', 'energii', '...","['Fotosyntéza je proces, při kterém rostliny v...",sport - sportem - oblíbeným - energii - fotosy...,False


In [33]:
docs = df_input['Document']
y = df_input['Topic']

**Set up model and train**

In [34]:
# Skip over dimensionality reduction, replace cluster model with classifier,
# and reduce frequent words while we are at it.
empty_dimensionality_model = BaseDimensionalityReduction()
clf = LogisticRegression()
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)

In [35]:
# Create a fully supervised BERTopic instance
topic_model= BERTopic(
        embedding_model = czech_model,
        umap_model = empty_dimensionality_model,
        hdbscan_model = clf,
        ctfidf_model = ctfidf_model
)

topics, probs = topic_model.fit_transform(docs, y=y)

In [36]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,32,0_film_filmová_filmy_režisér,"[film, filmová, filmy, režisér, získal, filmu,...","[Český film ""Kolya"" získal Oskara za nejlepší ..."
1,1,30,1_reakce_chemická_chemické_nebo,"[reakce, chemická, chemické, nebo, atomy, před...",[Chemické reakce mohou být exotermické nebo en...
2,2,26,2_sportem_je_sport_který,"[sportem, je, sport, který, pro, vyžaduje, cyk...",[Cyklistika je oblíbeným sportem pro rekreační...
3,3,22,3_revoluce_válka_měla_velká,"[revoluce, válka, měla, velká, spojených, svou...",[Dobyvatel Alexandr Veliký rozšířil svou říši ...
4,4,21,4_prostředí_proces_systému_organismů,"[prostředí, proces, systému, organismů, organi...",[Ekosystémy jsou vzájemně propojené sítě živýc...


**Assign original topic labels to the classification topics**

In [37]:
# Map input `y` to topics
mappings = topic_model.topic_mapper_.get_mappings()


#y_mapped = [mappings[val] for val in y]
#mappings = {value: data["target_names"][key] for key, value in mappings.items()}

In [38]:
mappings

{0: 4, 1: 3, 2: 2, 3: 0, 4: 1}

In [39]:
# Assign original classes to our topics
df = topic_model.get_topic_info()

In [40]:
df["Class"] = df.Topic.map(mappings)
df

,Topic,Count,Name,Representation,Representative_Docs,Class
0,0,32,0_film_filmová_filmy_režisér,"[film, filmová, filmy, režisér, získal, filmu,...","[Český film ""Kolya"" získal Oskara za nejlepší ...",4
1,1,30,1_reakce_chemická_chemické_nebo,"[reakce, chemická, chemické, nebo, atomy, před...",[Chemické reakce mohou být exotermické nebo en...,3
2,2,26,2_sportem_je_sport_který,"[sportem, je, sport, který, pro, vyžaduje, cyk...",[Cyklistika je oblíbeným sportem pro rekreační...,2
3,3,22,3_revoluce_válka_měla_velká,"[revoluce, válka, měla, velká, spojených, svou...",[Dobyvatel Alexandr Veliký rozšířil svou říši ...,0
4,4,21,4_prostředí_proces_systému_organismů,"[prostředí, proces, systému, organismů, organi...",[Ekosystémy jsou vzájemně propojené sítě živýc...,1


**Define unlabeled documents**

In [41]:
new_docs = [
            'Revoluce byla v roce 1852', 
            'Chemická reakce je exotermni',
            'Buňky jsou v biologii velice známé',
            'Historie se opakuje v podobě válek a bitev',
            'Hokej je krásný sport',
            'Filmový festival je o zajímavých filmech a hercích',
            'Napoleon Bonaparte prohrál v bitvě u Waterloo'
            ]

**Run the classificatio on new data**

In [42]:
topic, _ = topic_model.transform(new_docs)


In [43]:
topic

[3, 1, 4, 3, 2, 0, 3]

**Write the results into a df**

In [44]:
df_results = pd.DataFrame()
index = 0

# Get information about the topics
for topic_id in topic:
    topic_info = topic_model.get_topic(topic_id)

    transformed_data = str(topic_info)

    df_results.loc[index, 'original sentence'] = new_docs[index]
    df_results.loc[index, 'topic number'] = topic_id
    df_results.loc[index, 'topic info'] = transformed_data
    
    index += 1

df_results.head(10)

,original sentence,topic number,topic info
0,Revoluce byla v roce 1852,3.0,"[('revoluce', 0.5507027873656173), ('válka', 0..."
1,Chemická reakce je exotermni,1.0,"[('reakce', 0.6109700011845425), ('chemická', ..."
2,Buňky jsou v biologii velice známé,4.0,"[('prostředí', 0.6132919248148784), ('proces',..."
3,Historie se opakuje v podobě válek a bitev,3.0,"[('revoluce', 0.5507027873656173), ('válka', 0..."
4,Hokej je krásný sport,2.0,"[('sportem', 0.6327541567889825), ('je', 0.605..."
5,Filmový festival je o zajímavých filmech a her...,0.0,"[('film', 0.5921687617146105), ('filmová', 0.5..."
6,Napoleon Bonaparte prohrál v bitvě u Waterloo,3.0,"[('revoluce', 0.5507027873656173), ('válka', 0..."


## Translate

In [45]:
""" from deep_translator import GoogleTranslator

def translate_sentences(sentences, source_language='cs', target_language='en'):
    translated_sentences = []
    
    for sentence in sentences:
        translated = GoogleTranslator(source=source_language, target=target_language).translate(sentence)
        translated_sentences.append(translated)
    
    return translated_sentences

# Example usage
sentences_to_translate = [
    'snažím se dovolat uz minut',
    'ahoj jak se mas',
    'mam se dobre a ty?'
]

source_language = 'cs'
target_language = 'en'

translated_sentences = translate_sentences(sentences_to_translate, source_language, target_language)

# Print the original and translated sentences
for original, translated in zip(sentences_to_translate, translated_sentences):
    print(f"Original: {original}")
    print(f"Translated: {translated}")
    print() """

' from deep_translator import GoogleTranslator\n\ndef translate_sentences(sentences, source_language=\'cs\', target_language=\'en\'):\n    translated_sentences = []\n    \n    for sentence in sentences:\n        translated = GoogleTranslator(source=source_language, target=target_language).translate(sentence)\n        translated_sentences.append(translated)\n    \n    return translated_sentences\n\n# Example usage\nsentences_to_translate = [\n    \'snažím se dovolat uz minut\',\n    \'ahoj jak se mas\',\n    \'mam se dobre a ty?\'\n]\n\nsource_language = \'cs\'\ntarget_language = \'en\'\n\ntranslated_sentences = translate_sentences(sentences_to_translate, source_language, target_language)\n\n# Print the original and translated sentences\nfor original, translated in zip(sentences_to_translate, translated_sentences):\n    print(f"Original: {original}")\n    print(f"Translated: {translated}")\n    print() '